In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [2]:
# %pip install python-docx datasets ragas
# %pip install datasets

# RAG MODULE

In [3]:
%%time
import os
import re
import sys
import json
from typing import List, Dict, Any

from dotenv import load_dotenv
load_dotenv()

from docx import Document as DocxDocument

# RAG 모듈 import
MODULES_PATH = os.path.join(os.getcwd(), "chatbot_app", "modules")
sys.path.append(MODULES_PATH)
sys.path.append(os.getcwd())
from chatbot_app.modules.rag_module import create_pipeline, RAGConfig


def make_run_tag(cfg: RAGConfig) -> str:
    """파일명에 '튜닝한 값'이 보이도록 태그를 충분히 포함"""
    rerank_flag = "T" if getattr(cfg, "enable_rerank", False) else "F"

    # (선택) 존재할 수도/없을 수도 있는 필드들은 getattr로 안전하게 접근
    fusion = getattr(cfg, "hybrid_fusion", "rrf")
    dw = getattr(cfg, "hybrid_dense_weight", None)
    sw = getattr(cfg, "hybrid_sparse_weight", None)
    tr = getattr(cfg, "hybrid_sparse_title_ratio", None)
    rrf_k = getattr(cfg, "rrf_k", None)

    bm25_alg = getattr(cfg, "bm25_algorithm", None)
    bm25_k1 = getattr(cfg, "bm25_k1", None)
    bm25_b = getattr(cfg, "bm25_b", None)
    bm25_mdc = getattr(cfg, "bm25_max_doc_chars", None)

    # 소수점/긴 값들 보기 좋게 축약
    def ffloat(x, nd=2):
        if x is None:
            return "NA"
        try:
            return f"{float(x):.{nd}f}"
        except Exception:
            return str(x)

    tag = (
        f"k{cfg.k_law}-{cfg.k_rule}-{cfg.k_case}"
        f"_m{cfg.search_multiplier}"
        f"_fu{fusion}"
        f"_w{ffloat(dw)}-{ffloat(sw)}"
        f"_tr{ffloat(tr)}"
        f"_rrf{rrf_k if rrf_k is not None else 'NA'}"
        f"_bm25{bm25_alg if bm25_alg is not None else 'NA'}"
        f"-k1{ffloat(bm25_k1)}-b{ffloat(bm25_b)}-mdc{bm25_mdc if bm25_mdc is not None else 'NA'}"
        f"_rerank{rerank_flag}"
    )
    # 파일명 안전 처리(공백/특수문자 제거)
    tag = re.sub(r"[^0-9A-Za-z_\-\.]+", "", tag)
    return tag


DOC_PATH = "./data/RAGAS/RAGAS_질문_10개_정리본.docx"


def parse_ragas_docx(doc_path: str) -> List[Dict[str, Any]]:
    doc = DocxDocument(doc_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    text = "\n".join(paras)

    blocks = re.split(r"\n(?=\d+\.\s)", text)

    items = []
    for b in blocks:
        b = b.strip()
        if not re.match(r"^\d+\.\s", b):
            continue

        parts = b.split("✔️ 모범답안", 1)
        if len(parts) != 2:
            continue

        q_part = re.sub(r"^\d+\.\s*", "", parts[0].strip())
        a_part = parts[1].strip()

        items.append({"question": q_part.strip(), "ground_truth": a_part.strip()})

    if len(items) == 0:
        raise ValueError("docx에서 문항을 1개도 파싱하지 못했습니다. 문서 포맷을 확인하세요.")
    return items


def main():
    # 0) 환경변수 체크
    required_env = ["PINECONE_API_KEY", "UPSTAGE_API_KEY", "OPENAI_API_KEY"]
    missing = [k for k in required_env if not os.getenv(k)]
    if missing:
        raise EnvironmentError(f"필수 환경변수가 없습니다: {missing}")

    # 1) 질문 로드
    items = parse_ragas_docx(DOC_PATH)
    print(f"✅ 파싱된 문항 수: {len(items)}")

    # 2) 파이프라인 생성 (⭐ 튜닝 포인트: 아래 값들을 바꿔가며 실험)
    # - NOTE: 여기서 넘기는 값이 '최종 적용'입니다.
    #         rag_module.py 안에서 기본값을 바꿔도, 여기서 k_law=5로 주면 '5'가 적용됩니다.
    cfg = RAGConfig(
        # (1) Retrieval 양/후보풀
        k_law=7, # 5
        k_rule=7, # 5
        k_case=3, # 3
        search_multiplier=4, # 2

        # (2) Rerank (retrieval 튜닝만 보면 False 권장)
        enable_rerank=True,

        # (3) Hybrid / RRF (rag_module.py의 RAGConfig에 해당 필드가 있을 때만 동작)
        hybrid_fusion="rrf",
        hybrid_dense_weight=0.5, # 0.6
        hybrid_sparse_weight=0.5, # 0.4
        hybrid_sparse_title_ratio=0.35, # 0.35
        rrf_k=60, # 60

        # (4) BM25 (rag_module.py의 RAGConfig에 해당 필드가 있을 때만 동작)
        bm25_algorithm="okapi",   # okapi / plus
        bm25_k1=1.8, # 1.5
        bm25_b=0.75, # 0.75
        bm25_max_doc_chars=3000,
        enable_bm25_title=True,
    )

    RUN_TAG = make_run_tag(cfg)
    OUT_JSONL = f"./data/RAGAS/rag_run_cache_{RUN_TAG}.jsonl"

    pipeline = create_pipeline(config=cfg)

    # 3) 출력 폴더 보장
    os.makedirs(os.path.dirname(OUT_JSONL), exist_ok=True)

    # 4) 캐시 파일로 저장 (jsonl: 한 줄에 한 샘플)
    #    - 각 row에 run_config를 같이 넣어서, 파일 내부만 봐도 어떤 튜닝인지 바로 알 수 있게 함
    #    - (주의) RAGAS 입력으로 쓸 때 run_config 컬럼은 무시되거나 제거해도 됨
    try:
        cfg_dict = cfg.model_dump()  # pydantic v2
    except Exception:
        try:
            cfg_dict = cfg.dict()     # pydantic v1
        except Exception:
            cfg_dict = {k: getattr(cfg, k) for k in dir(cfg) if not k.startswith("_")}

    with open(OUT_JSONL, "w", encoding="utf-8") as f:
        for i, ex in enumerate(items, start=1):
            q = ex["question"]
            gt = ex["ground_truth"]

            trace = pipeline.answer_with_trace(q, skip_normalization=False)
            answer = trace.get("answer", "") or ""
            docs = trace.get("docs", []) or []

            contexts = []
            for d in docs:
                try:
                    contexts.append(d.page_content)
                except Exception:
                    pass

            row = {
                "id": i,
                "question": q,
                "answer": answer,
                "contexts": contexts,
                "reference": gt,               # RAGAS가 요구하던 reference
                "ground_truths": [gt],         # 다른 지표용으로도 유지
                "normalized_query": trace.get("normalized_query", ""),

                # ✅ 튜닝 추적용(파일 내부에도 남김)
                "run_tag": RUN_TAG,
                "run_config": cfg_dict,
            }
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

            if i % 5 == 0:
                print(f"… {i}/{len(items)} 저장 완료")

    print(f"\n✅ RAG 실행 결과 저장 완료: {OUT_JSONL}")
    print("✅ RUN_TAG:", RUN_TAG)


if __name__ == "__main__":
    main()

✅ 파싱된 문항 수: 10


2026-01-30 17:34:13,989 - chatbot_app.modules.rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...
2026-01-30 17:34:16,210 - chatbot_app.modules.rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-01-30 17:34:16,261 - chatbot_app.modules.rag_module - INFO - ℹ️ SimpleTokenizer 사용 (BM25)
2026-01-30 17:34:17,117 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:34:17,117 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?
2026-01-30 17:34:17,134 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?'
2026-01-30 17:34:17,596 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:34:19,728 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:34:22,104 - httpx - INFO - HTTP Request: POST https://api.upstag

2026-01-30 17:35:45,571 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


… 5/10 저장 완료


2026-01-30 17:35:46,559 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:35:46,563 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 경기도에 사는데 임차주택이 경매절차로 넘어가면, 임대차보증금 일부라도 우선변제권으로 받을 수 있나요? (경매: 경매절차, 보증금: 임대차보증금, 돈먼저받기: 우선변제권)
2026-01-30 17:35:46,564 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='경기도에 사는데 임차주택이 경매절차로 넘어가면, 임대차보증금 일부라도 우선변제권으로 받을 수 있나요? (경매: 경매절차, 보증금: 임대차보증금, 돈먼저받기: 우선변제권)'
2026-01-30 17:35:47,059 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:35:47,905 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:35:48,815 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:35:49,941 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-01-30 17:35:49,965 - chatbot_app

2026-01-30 17:37:09,827 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차(경매) 매각기일이 잡혔습니다. 이 경우에도 경매절차(경매)를 멈출 수 있는 방법이 있나요?  

변경된 질문:  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차(경매) 매각기일이 잡혔습니다. 이 경우에도 경매절차(경매)를 멈출 수 있는 방법이 있나요?  

※ 참고: '전세사기'는 용어 사전에 '전세피해'로 직접 매핑되지 않았으나, 문맥상 '권리리스크'로 대체 가능한 것으로 판단하였습니다. 정확한 법률 용어 적용을 위해 추가 확인이 필요할 수 있습니다.  

(최종 출력 시 위 참고 문구 제외)  

최종 출력:  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차(경매) 매각기일이 잡혔습니다. 이 경우에도 경매절차(경매)를 멈출 수 있는 방법이 있나요?
2026-01-30 17:37:09,828 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차(경매) 매각기일이 잡혔습니다. 이 경우에도 경매절차(경매)를 멈출 수 있는 방법이 있나요?  

변경된 질문:  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차(경매) 매각기일이 잡혔습니다. 이 경우에도 경매절차(경매)를 멈출 수 있는 방법이 있나요?  

※ 참고: '전세사기'는 용어 사전에 '전세피해'로 직접 매핑되지 않았으나, 문맥상 '권리리스크'로 대체 가능한 것으로 판단하였습니다. 정확한 법률 용어 적용을 위해 추가 확인이 필요할 수 있습니다.  

(최종 출력 시 위 참고 문구 제외)  

최종 출력:  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차(경매) 매각기일이 잡혔습니다. 이 경우에도 경매절차(경매)를 멈출 수 있는 방법이 있나요?'
2026-01-

… 10/10 저장 완료

✅ RAG 실행 결과 저장 완료: ./data/RAGAS/rag_run_cache_k7-7-3_m4_furrf_w0.50-0.50_tr0.35_rrf60_bm25okapi-k11.80-b0.75-mdc3000_rerankT.jsonl
✅ RUN_TAG: k7-7-3_m4_furrf_w0.50-0.50_tr0.35_rrf60_bm25okapi-k11.80-b0.75-mdc3000_rerankT
CPU times: total: 5.73 s
Wall time: 3min 12s


# RAGAS

## 테스트

In [4]:
# import ragas
# print("RAGAS version:", ragas.__version__)

# print("\n🔍 Testing ContextRecall import methods:")

# # 방법 1: collections (권장)
# try:
#     from ragas.metrics.collections import ContextRecall
#     print("✅ collections import works:", ContextRecall)
# except Exception as e:
#     print("❌ collections import failed:", e)

# # 방법 2: ragas.metrics (deprecated 가능)
# try:
#     from ragas.metrics import ContextRecall
#     print("✅ ragas.metrics import works (deprecated likely):", ContextRecall)
# except Exception as e:
#     print("❌ ragas.metrics import failed:", e)

# # 방법 3: 내부 경로(비권장)
# try:
#     from ragas.metrics._context_recall import ContextRecall as _ContextRecall
#     print("✅ internal import works (not recommended):", _ContextRecall)
# except Exception as e:
#     print("❌ internal import failed:", e)


In [5]:
# %%time
# import ragas
# print(f"RAGAS version: {ragas.__version__}")

# # 어떤 import 방법이 작동하는지 확인
# print("\n🔍 Testing import methods:")

# # 방법 1: 내부 클래스 직접 import (가장 확실)
# try:
#     from ragas.metrics._answer_relevance import AnswerRelevancy
#     print("✅ Method 1 works: from ragas.metrics._answer_relevance import AnswerRelevancy")
#     print(f"   Type: {type(AnswerRelevancy)}")
# except Exception as e:
#     print(f"❌ Method 1 failed: {e}")

# # 방법 2: ragas.metrics에서 대문자 클래스 import
# try:
#     from ragas.metrics import AnswerRelevancy
#     print("✅ Method 2 works: from ragas.metrics import AnswerRelevancy")
#     print(f"   Type: {type(AnswerRelevancy)}")
# except Exception as e:
#     print(f"❌ Method 2 failed: {e}")

# # 방법 3: deprecated 방법
# try:
#     from ragas.metrics import answer_relevancy
#     print(f"✅ Method 3: from ragas.metrics import answer_relevancy")
#     print(f"   Type: {type(answer_relevancy)}")
#     # 이게 클래스인지 확인
#     if hasattr(answer_relevancy, '__call__'):
#         print("   → This is callable (good!)")
#     else:
#         print("   → This is a module (bad!)")
# except Exception as e:
#     print(f"❌ Method 3 failed: {e}")

In [6]:
# import ragas.metrics.collections as c
# print([name for name in dir(c) if "Context" in name and "Precision" in name or "Recall" in name])

In [7]:
# from ragas.metrics.collections import ContextRecall
# print("ContextRecall symbol =", ContextRecall)
# print("type(ContextRecall) =", type(ContextRecall))

## 공통

In [8]:
%%time
import os
import json
import glob
import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv
import gc  # 메모리 관리
load_dotenv()

from openai import OpenAI
from ragas.llms import llm_factory
from langchain_openai import OpenAIEmbeddings 
from langchain_upstage import ChatUpstage, UpstageEmbeddings

openai_client = OpenAI()
ragas_llm = llm_factory(
    model="gpt-4o-mini",
    provider="openai",
    client=openai_client,
)

# ragas_embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     openai_api_key=os.getenv("OPENAI_API_KEY")
# )

# ✅ 평가용 임베딩도 Upstage로 (OpenAI 비용 제거)
# - 권장 모델명: "solar-embedding-1-large"
# - UpstageEmbeddings가 내부적으로 query/passage suffix를 자동 처리하는 구현이 일반적입니다. :contentReference[oaicite:1]{index=1}
ragas_embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

print("✅ ragas_llm ready:", type(ragas_llm))
print("✅ ragas_embeddings ready:", type(ragas_embeddings))

# ✅ embedding 테스트
try:
    test_vec = ragas_embeddings.embed_query("테스트")
    print(f"✅ Embedding test successful! Dimension: {len(test_vec)}")
except Exception as e:
    print(f"❌ Embedding test failed: {e}")


# 최신 캐시 jsonl 찾기
def find_latest_cache(pattern="./data/RAGAS/rag_run_cache_*.jsonl") -> str:
    candidates = glob.glob(pattern)
    if not candidates:
        raise FileNotFoundError(f"캐시 파일이 없습니다: {pattern}")
    return max(candidates, key=os.path.getmtime)

CACHE_JSONL = find_latest_cache()
print("✅ latest cache:", CACHE_JSONL)

# jsonl 로드
def load_jsonl(path: str):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                rows.append(json.loads(line))
    return rows

raw_rows = load_jsonl(CACHE_JSONL)

# 컬럼명 표준화 + 데이터 검증 강화
def normalize_row_schema(rows):
    out = []
    for i, r in enumerate(rows):
        rr = dict(r)

        # question/user_input 통일
        if "question" not in rr and "user_input" in rr:
            rr["question"] = rr["user_input"]
        
        # ✅ question이 비어있거나 None이면 스킵
        if not rr.get("question"):
            print(f"⚠️ Warning: Row {i} has empty question, skipping")
            continue

        # ✅ answer가 비어있거나 None이면 기본값 설정
        if not rr.get("answer"):
            print(f"⚠️ Warning: Row {i} has empty answer, using default")
            rr["answer"] = "답변을 생성할 수 없습니다."

        # contexts 통일
        if "contexts" not in rr and "retrieved_contexts" in rr:
            rr["contexts"] = rr["retrieved_contexts"]
        
        # ✅ contexts가 없으면 빈 리스트로 초기화
        if "contexts" not in rr:
            rr["contexts"] = []
        
        # ✅ contexts가 리스트가 아니면 변환
        if not isinstance(rr.get("contexts"), list):
            rr["contexts"] = [str(rr["contexts"])] if rr["contexts"] else []

        # reference 통일
        if "reference" not in rr:
            gts = rr.get("ground_truths", None)
            if isinstance(gts, list) and len(gts) > 0:
                rr["reference"] = gts[0]
            elif "ground_truth" in rr:
                rr["reference"] = rr["ground_truth"]
            else:
                rr["reference"] = ""  # ✅ 기본값 설정

        out.append(rr)
    return out

rows = normalize_row_schema(raw_rows)
print(f"✅ Normalized rows: {len(rows)}")

# ✅ 데이터 검증 함수 추가
def validate_data(rows, required_fields):
    """데이터 검증 및 문제 리포트"""
    issues = []
    for i, r in enumerate(rows):
        for field in required_fields:
            if field not in r:
                issues.append(f"Row {i}: missing field '{field}'")
            elif r[field] is None:
                issues.append(f"Row {i}: field '{field}' is None")
            elif field in ["question", "answer"] and not str(r[field]).strip():
                issues.append(f"Row {i}: field '{field}' is empty")
    
    if issues:
        print("⚠️ Data validation issues:")
        for issue in issues[:10]:  # 처음 10개만 출력
            print(f"  - {issue}")
        if len(issues) > 10:
            print(f"  ... and {len(issues) - 10} more issues")
    else:
        print("✅ Data validation passed")
    
    return len(issues) == 0

# ✅ 컨텍스트 클립 개선
def clip_rows(rows, *, max_ctx=3, max_chars=600, clip_reference_chars=None, clip_answer_chars=None):
    clipped = []
    for r in rows:
        rr = dict(r)

        ctx = rr.get("contexts", []) or []
        # ✅ None 값 필터링 추가
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]

        if clip_reference_chars is not None and rr.get("reference"):
            rr["reference"] = str(rr["reference"])[:clip_reference_chars]

        if clip_answer_chars is not None and rr.get("answer"):
            rr["answer"] = str(rr["answer"])[:clip_answer_chars]

        clipped.append(rr)
    return clipped

# ✅ 저장 폴더
OUT_DIR = "./data/RAGAS"
os.makedirs(OUT_DIR, exist_ok=True)

def save_result(result, metric_name: str) -> str:
    """
    result: ragas evaluate result
    metric_name: 파일명에 박을 이름
    """
    df = result.to_pandas()
    
    # ✅ 결과 요약 출력
    print(f"\n📊 {metric_name} Results:")
    print(f"  Mean: {df[metric_name].mean():.4f}")
    print(f"  Std: {df[metric_name].std():.4f}")
    print(f"  Min: {df[metric_name].min():.4f}")
    print(f"  Max: {df[metric_name].max():.4f}")
    print(f"  NaN count: {df[metric_name].isna().sum()}")
    
    out_path = os.path.join(OUT_DIR, f"ragas_result_{metric_name}.csv")
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ saved: {out_path}\n")
    return out_path

# ✅ 메모리 정리 함수
def cleanup_memory():
    """메모리 정리"""
    gc.collect()
    print("🧹 Memory cleaned")

✅ ragas_llm ready: <class 'ragas.llms.base.InstructorLLM'>
✅ ragas_embeddings ready: <class 'langchain_upstage.embeddings.UpstageEmbeddings'>


2026-01-30 17:37:26,107 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"


✅ Embedding test successful! Dimension: 4096
✅ latest cache: ./data/RAGAS\rag_run_cache_k7-7-3_m4_furrf_w0.50-0.50_tr0.35_rrf60_bm25okapi-k11.80-b0.75-mdc3000_rerankT.jsonl
✅ Normalized rows: 10
CPU times: total: 1.17 s
Wall time: 2.38 s


## context_precision

In [9]:
%%time
from ragas import evaluate
from ragas.metrics._context_precision import ContextPrecision

print("📋 Validating data for context_precision...")
validate_data(rows, required_fields=["question", "contexts"])

rows_cp = clip_rows(rows, max_ctx=3, max_chars=600)

eval_ds_cp = Dataset.from_list(rows_cp)

print("▶ context_precision evaluate start")
try:
    res_cp = evaluate(
        dataset=eval_ds_cp,
        metrics=[ContextPrecision()],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        show_progress=True,
        raise_exceptions=True, 
        batch_size=1,
    )
    print("✅ context_precision done:", res_cp)
    cp_csv = save_result(res_cp, "context_precision")
except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

cleanup_memory()

📋 Validating data for context_precision...
✅ Data validation passed
▶ context_precision evaluate start


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Batch 1/10:   0%|          | 0/1 [00:00<?, ?it/s]

2026-01-30 17:37:29,096 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:31,505 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:34,663 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:38,265 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:41,476 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:44,252 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:47,273 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:49,745 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:37:52,163 - httpx 

✅ context_precision done: {'context_precision': 1.0000}

📊 context_precision Results:
  Mean: 1.0000
  Std: 0.0000
  Min: 1.0000
  Max: 1.0000
  NaN count: 0
✅ saved: ./data/RAGAS\ragas_result_context_precision.csv

🧹 Memory cleaned
CPU times: total: 2.34 s
Wall time: 1min 34s


## context_recall

In [10]:
%%time
from ragas import evaluate
from ragas.metrics._context_recall import ContextRecall

print("📋 Validating data for context_recall...")
validate_data(rows, required_fields=["question", "contexts", "reference"])

# 너무 무거우면 샘플만
SAMPLE_SIZE = 10  
rows_cr = clip_rows(
    rows[:SAMPLE_SIZE],  
    max_ctx=2,
    max_chars=200,  
    clip_reference_chars=300, 
    clip_answer_chars=300,  # optional
)

eval_ds_cr = Dataset.from_list(rows_cr)

print(f"▶ context_recall evaluate start (샘플 {SAMPLE_SIZE}개)")
try:
    metric_cr = ContextRecall()  
    print("metric_cr =", metric_cr, "| type =", type(metric_cr))

    res_cr = evaluate(
        dataset=eval_ds_cr,
        metrics=[metric_cr],          
        llm=ragas_llm,                
        embeddings=ragas_embeddings,  
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    print("✅ context_recall done:", res_cr)
    cr_csv = save_result(res_cr, "context_recall")
except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

cleanup_memory()

📋 Validating data for context_recall...
✅ Data validation passed
▶ context_recall evaluate start (샘플 10개)
metric_cr = ContextRecall(_required_columns={<MetricType.SINGLE_TURN: 'single_turn'>: {'reference', 'user_input', 'retrieved_contexts'}}, name='context_recall', llm=None, output_type=<MetricOutputType.CONTINUOUS: 'continuous'>, context_recall_prompt=ContextRecallClassificationPrompt(instruction=Given a context, and an answer, analyze each sentence in the answer and classify if the sentence can be attributed to the given context or not. Use only 'Yes' (1) or 'No' (0) as a binary classification. Output json with reason., examples=[(QCA(question='What can you tell me about albert Albert Einstein?', context="Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics,

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Batch 1/10:   0%|          | 0/1 [00:00<?, ?it/s]

2026-01-30 17:39:09,221 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:39:18,633 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:39:29,288 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:39:39,069 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:39:47,178 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:39:58,662 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:40:09,706 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:40:19,633 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:40:27,538 - httpx 

✅ context_recall done: {'context_recall': 0.7463}

📊 context_recall Results:
  Mean: 0.7463
  Std: 0.2402
  Min: 0.2222
  Max: 1.0000
  NaN count: 0
✅ saved: ./data/RAGAS\ragas_result_context_recall.csv

🧹 Memory cleaned
CPU times: total: 922 ms
Wall time: 1min 34s


## answer_relevancy

In [11]:
# %%time
# from ragas import evaluate
# from ragas.metrics._answer_relevance import AnswerRelevancy

# print("📋 Validating data for answer_relevancy...")
# validate_data(rows, required_fields=["question", "answer", "contexts"])

# rows_ar = clip_rows(rows, max_ctx=0, max_chars=0, clip_answer_chars=1000)
# for r in rows_ar:
#     r["contexts"] = []

# eval_ds_ar = Dataset.from_list(rows_ar)

# print("\n▶ answer_relevancy evaluate start")
# try:
#     res_ar = evaluate(
#         dataset=eval_ds_ar,
#         metrics=[AnswerRelevancy()],
#         llm=ragas_llm,
#         embeddings=ragas_embeddings,  # ✅ LangChain embeddings
#         show_progress=True,
#         raise_exceptions=True,
#         batch_size=1,
#     )
#     print("✅ answer_relevancy done:", res_ar)
#     ar_csv = save_result(res_ar, "answer_relevancy")
# except Exception as e:
#     print(f"❌ Error: {str(e)}")
#     import traceback
#     traceback.print_exc()

# cleanup_memory()

## faithfulness

In [12]:
# %%time
# from ragas import evaluate
# from ragas.metrics._faithfulness import Faithfulness

# print("📋 Validating data for faithfulness...")
# validate_data(rows, required_fields=["question", "answer", "contexts"])

# SAMPLE_SIZE = 5  # ✅ 5개만 추천
# rows_fa = clip_rows(
#     rows[:SAMPLE_SIZE],
#     max_ctx=2,
#     max_chars=400,  # ✅ 늘림
#     clip_answer_chars=800  # ✅ 늘림
# )

# eval_ds_fa = Dataset.from_list(rows_fa)

# print(f"▶ faithfulness evaluate start (샘플 {SAMPLE_SIZE}개)")
# try:
#     res_fa = evaluate(
#         dataset=eval_ds_fa,
#         metrics=[Faithfulness()],
#         llm=ragas_llm,
#         embeddings=ragas_embeddings,
#         show_progress=True,
#         raise_exceptions=True,
#         batch_size=1,
#     )
#     print("✅ faithfulness done:", res_fa)
#     fa_csv = save_result(res_fa, "faithfulness_sample")
# except Exception as e:
#     print(f"❌ Error: {str(e)}")
#     import traceback
#     traceback.print_exc()

# cleanup_memory()

# 결과 합산
## 전체

In [13]:
# # =========================
# # [셀 6/6] 마지막: 결과 합산(가능한 지표 자동 병합) + 총점 계산 + 저장
# # =========================
# import pandas as pd

# # 각 결과 CSV를 다시 로드
# df_cp = pd.read_csv(cp_csv)  # contains context_precision
# df_cr = pd.read_csv(cr_csv)  # contains context_recall
# df_ar = pd.read_csv(ar_csv# =========================
# # [셀 6/6] 마지막: 결과 합산(존재하는 지표만 자동 병합) + 총점 계산 + 저장
# # - 어떤 지표를 일부만 돌려도(예: answer_relevancy만) 에러 없이 합산되도록 수정
# # - ./data/RAGAS 폴더의 ragas_result_*.csv 를 자동 탐색
# # =========================
# import os
# import glob
# import pandas as pd

# # ✅ 저장 폴더 (공통부분에서 OUT_DIR를 만들었다면 그대로 사용, 없으면 기본값)
# OUT_DIR = globals().get("OUT_DIR", "./data/RAGAS")
# os.makedirs(OUT_DIR, exist_ok=True)

# # ✅ 유틸: 파일 존재 체크 + 로드
# def safe_read_csv(path: str):
#     if path and os.path.exists(path):
#         return pd.read_csv(path)
#     return None

# # ✅ 유틸: id 없으면 생성 (merge용)
# def ensure_id(df: pd.DataFrame):
#     if df is None:
#         return None
#     if "id" not in df.columns:
#         df = df.copy()
#         df["id"] = range(1, len(df) + 1)
#     return df

# # ✅ (권장) 이전 셀에서 cp_csv/cr_csv/ar_csv/fa_csv 변수가 있으면 그대로 사용
# cp_path = globals().get("cp_csv", None)
# cr_path = globals().get("cr_csv", None)
# ar_path = globals().get("ar_csv", None)
# fa_path = globals().get("fa_csv", None)

# # ✅ 없으면 폴더에서 자동 탐색 (가장 최근 파일 선택)
# def pick_latest(pattern: str):
#     cands = glob.glob(os.path.join(OUT_DIR, pattern))
#     if not cands:
#         return None
#     return max(cands, key=os.path.getmtime)

# if cp_path is None:
#     cp_path = pick_latest("ragas_result_context_precision*.csv")
# if cr_path is None:
#     cr_path = pick_latest("ragas_result_context_recall*.csv")
# if ar_path is None:
#     ar_path = pick_latest("ragas_result_answer_relevancy*.csv")
# if fa_path is None:
#     fa_path = pick_latest("ragas_result_faithfulness*.csv")

# print("🔎 Using result files:")
# print(" - context_precision:", cp_path)
# print(" - context_recall   :", cr_path)
# print(" - answer_relevancy :", ar_path)
# print(" - faithfulness     :", fa_path)

# df_cp = ensure_id(safe_read_csv(cp_path))
# df_cr = ensure_id(safe_read_csv(cr_path))
# df_ar = ensure_id(safe_read_csv(ar_path))
# df_fa = ensure_id(safe_read_csv(fa_path))

# # ✅ 기준 DF 만들기: "가장 확실히 존재하는 것"부터 시작
# # 우선순위: cp > cr > ar > fa
# df_all = None
# for candidate in [df_cp, df_cr, df_ar, df_fa]:
#     if candidate is not None:
#         df_all = candidate.copy()
#         break

# if df_all is None:
#     raise FileNotFoundError(f"결과 CSV를 하나도 찾지 못했습니다. OUT_DIR={OUT_DIR} 내 ragas_result_*.csv 를 확인하세요.")

# # ✅ 필요한 컬럼만 안전하게 merge
# def safe_merge(base: pd.DataFrame, other: pd.DataFrame, cols: list):
#     if other is None:
#         return base
#     keep = [c for c in (["id"] + cols) if c in other.columns]
#     if "id" not in keep:
#         return base
#     return base.merge(other[keep], on="id", how="left")

# df_all = safe_merge(df_all, df_cp, ["context_precision"])
# df_all = safe_merge(df_all, df_cr, ["context_recall"])
# df_all = safe_merge(df_all, df_ar, ["answer_relevancy"])
# df_all = safe_merge(df_all, df_fa, ["faithfulness"])

# # ✅ Retrieval score R (가능할 때만)
# if all(c in df_all.columns for c in ["context_recall", "context_precision"]):
#     df_all["R"] = 0.6 * df_all["context_recall"] + 0.4 * df_all["context_precision"]

# # ✅ Generation score G (가능할 때만)
# # - faithfulness가 있으면 함께 사용
# # - 없으면 answer_relevancy만 사용
# if "faithfulness" in df_all.columns and "answer_relevancy" in df_all.columns:
#     df_all["G"] = 0.6 * df_all["faithfulness"] + 0.4 * df_all["answer_relevancy"]
# elif "answer_relevancy" in df_all.columns:
#     df_all["G"] = df_all["answer_relevancy"]

# # ✅ Total (가능한 구성으로 자동 계산)
# # 기본 가중치:
# # - R(검색 품질) + G(답변 품질)
# # - R이 없으면 G만
# # - G도 없으면(이론상 없음) NaN
# if "R" in df_all.columns and "G" in df_all.columns:
#     df_all["Total"] = 0.55 * df_all["R"] + 0.45 * df_all["G"]
# elif "G" in df_all.columns:
#     df_all["Total"] = df_all["G"]

# # ✅ 저장
# out_path_all = os.path.join(OUT_DIR, "ragas_merged_all_scores.csv")
# df_all.to_csv(out_path_all, index=False, encoding="utf-8")
# print("✅ merged saved:", out_path_all)

# # ✅ 요약 출력
# summary_cols = [c for c in ["context_precision","context_recall","answer_relevancy","faithfulness","R","G","Total"] if c in df_all.columns]
# print("\n📌 mean summary")
# print(df_all[summary_cols].mean(numeric_only=True))
# )  # contains answer_relevancy
# # faithfulness는 샘플 결과라 전체 병합에 직접 쓰기 애매 (원하면 id로 merge 가능)
# df_fa = pd.read_csv(fa_csv)  # faithfulness_sample

# # 공통키: 보통 'id'가 있음. 없으면 index로 합침.
# def ensure_id(df):
#     if "id" not in df.columns:
#         df = df.copy()
#         df["id"] = range(1, len(df) + 1)
#     return df

# df_cp = ensure_id(df_cp)
# df_cr = ensure_id(df_cr)
# df_ar = ensure_id(df_ar)

# # 병합 (id 기준)
# df_all = df_cp.merge(df_cr[["id", "context_recall"]], on="id", how="left")
# df_all = df_all.merge(df_ar[["id", "answer_relevancy"]], on="id", how="left")

# # Retrieval score (항상 가능)
# if all(c in df_all.columns for c in ["context_recall", "context_precision"]):
#     df_all["R"] = 0.6 * df_all["context_recall"] + 0.4 * df_all["context_precision"]

# # -----------------------------
# # Case 1) faithfulness 있는 경우 (샘플이면 주의)
# # -----------------------------
# # df_fa는 샘플(SAMPLE_N)만 들어있을 수 있음 → 원하면 아래처럼 id로 병합
# df_fa = ensure_id(df_fa)
# if "faithfulness" in df_fa.columns:
#     df_all = df_all.merge(df_fa[["id", "faithfulness"]], on="id", how="left")

# # -----------------------------
# # Total 점수(2버전)
# # -----------------------------
# # (A) faithfulness가 있는 행만: G_with_faith / Total_with_faith
# if all(c in df_all.columns for c in ["faithfulness", "answer_relevancy", "R"]):
#     df_all["G_with_faith"] = 0.6 * df_all["faithfulness"] + 0.4 * df_all["answer_relevancy"]
#     df_all["Total_with_faith"] = 0.55 * df_all["R"] + 0.45 * df_all["G_with_faith"]

# # (B) faithfulness 없는 경우: answer_relevancy 단독 + Retrieval 비중↑
# # if all(c in df_all.columns for c in ["answer_relevancy", "R"]):
# #     df_all["G_no_faith"] = df_all["answer_relevancy"]
# #     df_all["Total_no_faith"] = 0.65 * df_all["R"] + 0.35 * df_all["G_no_faith"]

# # 저장
# out_path_all = os.path.join(OUT_DIR, "ragas_merged_all_scores.csv")
# df_all.to_csv(out_path_all, index=False, encoding="utf-8")
# print("✅ merged saved:", out_path_all)

# # 요약 출력
# summary_cols = [c for c in ["context_precision","context_recall","answer_relevancy","faithfulness","R","Total_with_faith","Total_no_faith"] if c in df_all.columns]
# print("\n📌 mean summary")
# print(df_all[summary_cols].mean(numeric_only=True))

## R

In [14]:
# matplotlib 없이: 질문별 recall/precision merge → R_i 계산 → 분포/요약/하위문항/recall=0 추출 + 저장

import os
import pandas as pd
import numpy as np

OUT_DIR = "./data/RAGAS"
recall_csv = os.path.join(OUT_DIR, "ragas_result_context_recall.csv")
prec_csv   = os.path.join(OUT_DIR, "ragas_result_context_precision.csv")

w_recall = 0.6
w_prec   = 0.4

# 1) 로드
df_r = pd.read_csv(recall_csv)
df_p = pd.read_csv(prec_csv)

# 2) id 보장
if "id" not in df_r.columns:
    df_r = df_r.copy()
    df_r["id"] = range(1, len(df_r) + 1)

if "id" not in df_p.columns:
    df_p = df_p.copy()
    df_p["id"] = range(1, len(df_p) + 1)

# 3) merge (필요 컬럼만)
keep_r = [c for c in ["id", "question", "context_recall"] if c in df_r.columns]
keep_p = [c for c in ["id", "question", "context_precision"] if c in df_p.columns]

df = df_r[keep_r].merge(df_p[keep_p], on="id", how="inner", suffixes=("_r", "_p"))

# question 정리
if "question_r" in df.columns and "question_p" in df.columns:
    df["question"] = df["question_r"].fillna(df["question_p"])
    df.drop(columns=["question_r", "question_p"], inplace=True)
elif "question_r" in df.columns:
    df.rename(columns={"question_r": "question"}, inplace=True)
elif "question_p" in df.columns:
    df.rename(columns={"question_p": "question"}, inplace=True)

# 4) R_i 계산
if "context_recall" not in df.columns or "context_precision" not in df.columns:
    raise ValueError("context_recall 또는 context_precision 컬럼이 없습니다. CSV 컬럼명을 확인하세요.")

df["R_i"] = w_recall * df["context_recall"] + w_prec * df["context_precision"]

# 5) 요약 통계
print("✅ Per-question summary (recall/precision/R_i)")
print(df[["context_recall", "context_precision", "R_i"]].describe())

# 6) R_i 분위수(분포 느낌)
qs = df["R_i"].quantile([0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
print("\n📌 R_i quantiles")
print(qs)

# 7) 텍스트 히스토그램(10 bins)
bins = np.linspace(df["R_i"].min(), df["R_i"].max(), 11)
counts, edges = np.histogram(df["R_i"], bins=bins)

print("\n📊 R_i histogram (10 bins)")
for i in range(len(counts)):
    lo = edges[i]
    hi = edges[i+1]
    bar = "█" * counts[i]
    print(f"{lo:0.3f} ~ {hi:0.3f} | {counts[i]:2d} {bar}")

# 8) 하위 문항
cols_show = ["id"]
if "question" in df.columns:
    cols_show.append("question")
cols_show += ["context_recall", "context_precision", "R_i"]

print("\n🔻 Lowest R_i (top 10)")
print(df.sort_values("R_i", ascending=True).head(10)[cols_show])

# 9) recall=0 문항
print("\n🚨 recall == 0 rows")
df_fail = df[df["context_recall"] == 0].copy()
if df_fail.empty:
    print("없음")
else:
    print(df_fail.sort_values("id")[cols_show])

# 10) 저장
out_path = os.path.join(OUT_DIR, "ragas_Ri_per_question.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
print("\n💾 saved:", out_path)


✅ Per-question summary (recall/precision/R_i)
       context_recall  context_precision        R_i
count       10.000000       1.000000e+01  10.000000
mean         0.746306       1.000000e+00   0.847784
std          0.240206       1.170278e-16   0.144123
min          0.222222       1.000000e+00   0.533333
25%          0.725000       1.000000e+00   0.835000
50%          0.828571       1.000000e+00   0.897143
75%          0.880952       1.000000e+00   0.928571
max          1.000000       1.000000e+00   1.000000

📌 R_i quantiles
0.00    0.533333
0.10    0.644762
0.25    0.835000
0.50    0.897143
0.75    0.928571
0.90    0.950909
1.00    1.000000
Name: R_i, dtype: float64

📊 R_i histogram (10 bins)
0.533 ~ 0.580 |  1 █
0.580 ~ 0.627 |  0 
0.627 ~ 0.673 |  1 █
0.673 ~ 0.720 |  0 
0.720 ~ 0.767 |  0 
0.767 ~ 0.813 |  0 
0.813 ~ 0.860 |  1 █
0.860 ~ 0.907 |  2 ██
0.907 ~ 0.953 |  4 ████
0.953 ~ 1.000 |  1 █

🔻 Lowest R_i (top 10)
   id  context_recall  context_precision       R_i
6   7        

## 이미지저장

In [15]:
import os
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

OUT_DIR = "./data/RAGAS"
os.makedirs(OUT_DIR, exist_ok=True)

recall_csv = os.path.join(OUT_DIR, "ragas_result_context_recall.csv")
prec_csv   = os.path.join(OUT_DIR, "ragas_result_context_precision.csv")

w_recall = 0.6
w_prec   = 0.4

# 1) load
df_r = pd.read_csv(recall_csv)
df_p = pd.read_csv(prec_csv)

# 2) ensure id
if "id" not in df_r.columns:
    df_r = df_r.copy()
    df_r["id"] = range(1, len(df_r) + 1)
if "id" not in df_p.columns:
    df_p = df_p.copy()
    df_p["id"] = range(1, len(df_p) + 1)

# 3) merge (필요 컬럼만)
keep_r = [c for c in ["id", "question", "context_recall"] if c in df_r.columns]
keep_p = [c for c in ["id", "question", "context_precision"] if c in df_p.columns]

df = df_r[keep_r].merge(df_p[keep_p], on="id", how="inner", suffixes=("_r", "_p"))

# question 정리
if "question_r" in df.columns and "question_p" in df.columns:
    df["question"] = df["question_r"].fillna(df["question_p"])
    df.drop(columns=["question_r", "question_p"], inplace=True)
elif "question_r" in df.columns:
    df.rename(columns={"question_r": "question"}, inplace=True)
elif "question_p" in df.columns:
    df.rename(columns={"question_p": "question"}, inplace=True)

# 4) compute R_i
if "context_recall" not in df.columns or "context_precision" not in df.columns:
    raise ValueError("CSV에 context_recall / context_precision 컬럼이 없습니다. 컬럼명을 확인하세요.")

df["R_i"] = w_recall * df["context_recall"] + w_prec * df["context_precision"]

# 5) round to 2 decimals (CSV/이미지 모두)
for c in ["context_recall", "context_precision", "R_i"]:
    df[c] = df[c].astype(float).round(2)

# 6) overall R (mean of R_i) -> CSV에만
R_overall = float(df["R_i"].mean().round(2))

# 7) save CSV (요약행 1줄 추가)
out_csv = os.path.join(OUT_DIR, "ragas_Ri_per_question.csv")

cols = ["id"]
if "question" in df.columns:
    cols.append("question")
cols += ["context_recall", "context_precision", "R_i"]

df_out = df[cols].copy()

summary_row = {c: "" for c in df_out.columns}
summary_row["id"] = "OVERALL"
summary_row["R_i"] = R_overall  # 전체 R 값은 여기서만 제공
df_out = pd.concat([df_out, pd.DataFrame([summary_row])], ignore_index=True)

df_out.to_csv(out_csv, index=False, encoding="utf-8")
print("✅ saved:", out_csv)
print("✅ R_overall (mean of R_i):", R_overall)

# ---------- Plotly -> PNG helper ----------
def save_png(fig, filename: str):
    path = os.path.join(OUT_DIR, filename)
    fig.write_image(path, scale=2)  # kaleido 필요
    print("🖼️ PNG saved:", path)

# 8) Bar PNG (질문 번호 고정: id 오름차순)

def short(s, n=40):
    if not isinstance(s, str):
        return ""
    s = s.replace("\n", " ")
    return s if len(s) <= n else s[:n] + "…"

df_plot = df.copy()

# 🔑 핵심: id 기준으로 정렬 (절대 R_i로 정렬하지 않음)
df_plot = df_plot.sort_values("id", ascending=True)

# 라벨 생성
if "question" in df_plot.columns:
    df_plot["label"] = df_plot.apply(
        lambda r: f'{int(r["id"])}. {short(r["question"])}',
        axis=1
    )
else:
    df_plot["label"] = df_plot["id"].astype(str)

fig_bar = px.bar(
    df_plot,
    x="R_i",
    y="label",
    orientation="h",
    title="R_i per Question (fixed order: Q1 → Q10)",
    text="R_i",
)

# 숫자 포맷 고정 (소수 2자리)
fig_bar.update_traces(
    texttemplate="%{text:.2f}",
    textposition="outside"
)

fig_bar.update_layout(
    xaxis=dict(range=[0, 1.05]),
    yaxis=dict(autorange="reversed"),  # 1번이 맨 위로 오게
    margin=dict(l=10, r=10, t=60, b=10),
    height=520
)

save_png(fig_bar, "R_i_top10_bar.png")


# 9) Table PNG (10개 전체)
table_df = df[cols].copy()
# 숫자 컬럼은 문자열로 2자리 고정 표기
for c in ["context_recall", "context_precision", "R_i"]:
    if c in table_df.columns:
        table_df[c] = table_df[c].map(lambda x: f"{float(x):.2f}")

fig_table = go.Figure(
    data=[
        go.Table(
            header=dict(values=list(table_df.columns)),
            cells=dict(values=[table_df[c].tolist() for c in table_df.columns]),
        )
    ]
)
fig_table.update_layout(title="Per-question metrics table (2 decimals)", height=420, margin=dict(l=10, r=10, t=60, b=10))

save_png(fig_table, "R_i_top10_table.png")

print("✅ done.")


✅ saved: ./data/RAGAS\ragas_Ri_per_question.csv
✅ R_overall (mean of R_i): 0.85


2026-01-30 17:40:36,083 - choreographer.browsers.chromium - INFO - Chromium init'ed with kwargs {}
2026-01-30 17:40:36,086 - choreographer.browsers.chromium - INFO - Found chromium path: C:\Program Files\Google\Chrome\Application\chrome.exe
2026-01-30 17:40:36,087 - choreographer.utils._tmpfile - INFO - Temp directory created: C:\Users\Admin\AppData\Local\Temp\tmp6y4d482x.
2026-01-30 17:40:36,088 - choreographer.browser_async - INFO - Opening browser.
2026-01-30 17:40:36,090 - choreographer.utils._tmpfile - INFO - Temp directory created: C:\Users\Admin\AppData\Local\Temp\tmpm9ouw7re.
2026-01-30 17:40:36,091 - choreographer.browsers.chromium - INFO - Temporary directory at: C:\Users\Admin\AppData\Local\Temp\tmpm9ouw7re
2026-01-30 17:40:36,364 - kaleido.kaleido - INFO - Conforming 1 to file:///C:/Users/Admin/AppData/Local/Temp/tmp6y4d482x/index.html
2026-01-30 17:40:36,365 - kaleido.kaleido - INFO - Waiting on all navigates
2026-01-30 17:40:37,272 - kaleido.kaleido - INFO - All navigates

🖼️ PNG saved: ./data/RAGAS\R_i_top10_bar.png


2026-01-30 17:40:37,828 - kaleido.kaleido - INFO - Conforming 1 to file:///C:/Users/Admin/AppData/Local/Temp/tmp_2mp7v77/index.html
2026-01-30 17:40:37,829 - kaleido.kaleido - INFO - Waiting on all navigates
2026-01-30 17:40:38,724 - kaleido.kaleido - INFO - All navigates done, putting them all in queue.
2026-01-30 17:40:38,726 - kaleido.kaleido - INFO - Getting tab from queue (has 1)
2026-01-30 17:40:38,727 - kaleido.kaleido - INFO - Got 1254
2026-01-30 17:40:38,728 - kaleido._kaleido_tab - INFO - Processing Per_question_metrics_table_2_decimals.png
2026-01-30 17:40:38,728 - kaleido._kaleido_tab - INFO - Sending big command for Per_question_metrics_table_2_decimals.png.
2026-01-30 17:40:38,792 - kaleido._kaleido_tab - INFO - Sent big command for Per_question_metrics_table_2_decimals.png.
2026-01-30 17:40:38,794 - kaleido.kaleido - INFO - Reloading tab 1254 before return.
2026-01-30 17:40:38,883 - kaleido.kaleido - INFO - Putting tab 1254 back (queue size: 0).
2026-01-30 17:40:38,884 -

🖼️ PNG saved: ./data/RAGAS\R_i_top10_table.png
✅ done.
